In [5]:
# main.py
import os
import logging
import pandas as pd

from geomeppy import IDF

# Geometry modules
from geomz.building import create_building_with_roof_type
from geomz.geometry_overrides_from_excel import read_geometry_overrides_excel

from geomz.override_geometry_lookup_from_excel import override_geometry_lookup_from_excel
from geomz.geometry_lookup import geometry_lookup as default_geometry_lookup

# main.py
import pprint
from geomz.override_geometry_lookup_from_excel import override_geometry_lookup_from_excel
from geomz.geometry_lookup import geometry_lookup as default_geometry_lookup
# from path.to.your.helper import save_geometry_lookup_to_pyfile  # import your function

def save_geometry_lookup_to_pyfile(geometry_lookup, out_py_file="geometry_lookup_autogen.py"):
    """
    Writes the provided geometry_lookup dictionary to a Python file,
    with valid Python syntax. That file can then be imported as a module
    to reuse or inspect the updated dictionary.

    Args:
      geometry_lookup (dict): The in-memory dictionary you want to save.
      out_py_file (str): The path/name of the Python file to create.
    """
    # Use 'pprint' to get a nicely formatted string representation of the dictionary
    pretty_dict_str = pprint.pformat(geometry_lookup, width=120, sort_dicts=False)

    # Write that out to a .py file with a variable assignment
    with open(out_py_file, "w", encoding="utf-8") as f:
        f.write("# This file was auto-generated from Excel overrides.\n")
        f.write("# You can import this file as a Python module to get 'geometry_lookup'.\n\n")
        f.write("geometry_lookup = ")
        f.write(pretty_dict_str)
        f.write("\n")

# ...
# after reading excel_rules:
override_geom_from_excel = True
geom_excel_path = r"D:\Documents\E_Plus_2029_py\geometry_lookup.xlsx"

excel_rules = read_geometry_overrides_excel(geom_excel_path)
override_geometry_lookup_from_excel(default_geometry_lookup, excel_rules)
print("[Global] geometry_lookup dictionary has been updated with Excel rules.")


# 1) Load Excel & override
if override_geom_from_excel and os.path.isfile(geom_excel_path):
    excel_rules = read_geometry_overrides_excel(geom_excel_path)
    if excel_rules:
        override_geometry_lookup_from_excel(default_geometry_lookup, excel_rules)
        print(f"[Global] geometry_lookup dictionary has been updated with Excel rules. (Loaded {len(excel_rules)} rules)")
        # 2) Now save it to a new .py file
        out_py = "geometry_lookup_autogen.py"
        save_geometry_lookup_to_pyfile(default_geometry_lookup, out_py)
        print(f"[Global] Wrote updated geometry lookup to {out_py}")
    else:
        print("[Global] Excel file found but no valid rows. Using default geometry_lookup.")
else:
    print("[Global] Using default geometry_lookup (no Excel override).")



[Global] geometry_lookup dictionary has been updated with Excel rules.
[Global] geometry_lookup dictionary has been updated with Excel rules. (Loaded 32 rules)
[Global] Wrote updated geometry lookup to geometry_lookup_autogen.py


# Geometry

In [4]:
import pandas as pd
from collections import defaultdict
import json
import re

# Path to your Excel file
excel_file_path = r"D:\Documents\E_Plus_2029_py\geometry_lookup.xlsx"

# Read the Excel file
try:
    df = pd.read_excel(excel_file_path)
except FileNotFoundError:
    print(f"Error: The file {excel_file_path} does not exist.")
    exit(1)
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit(1)

# Initialize the main dictionary
geometry_lookup = defaultdict(lambda: defaultdict(dict))

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    building_function = row['building_function']
    building_type = row['building_type']
    calibration_stage = row['calibration_stage'].strip()  # Remove any leading/trailing whitespace
    perimeter_depth_min = row['perimeter_depth_min']
    perimeter_depth_max = row['perimeter_depth_max']
    has_core_value = row['has_core_value']
    
    # Convert has_core_value to boolean if it's not already
    if isinstance(has_core_value, str):
        has_core = has_core_value.strip().lower() == 'true'
    else:
        has_core = bool(has_core_value)
    
    # Define the calibration data
    calibration_data = {
        "perimeter_depth_range": (perimeter_depth_min, perimeter_depth_max),  # Using tuple
        "has_core": has_core
    }
    
    # Assign the calibration data to the appropriate place in the dictionary
    geometry_lookup[building_function][building_type][calibration_stage] = calibration_data

# Convert defaultdict to regular dict for serialization
geometry_lookup = dict(geometry_lookup)

# Optionally, sort the dictionary for better readability
def sort_dict(d):
    if isinstance(d, dict):
        return {k: sort_dict(v) for k, v in sorted(d.items())}
    return d

geometry_lookup = sort_dict(geometry_lookup)

# Custom function to serialize the dictionary with double quotes and proper booleans
def serialize_dict(d, indent=0):
    indent_space = '    ' * indent
    if isinstance(d, dict):
        items = []
        for i, (k, v) in enumerate(d.items()):
            key = f'"{k}"'
            value = serialize_dict(v, indent + 1)
            items.append(f'{indent_space}    {key}: {value}')
        return "{\n" + ",\n".join(items) + f"\n{indent_space}}}"
    elif isinstance(d, list):
        items = [serialize_dict(item, indent + 1) for item in d]
        return "[ " + ", ".join(items) + " ]"
    elif isinstance(d, tuple):
        items = ", ".join([serialize_dict(item, indent) for item in d])
        return f"({items})"
    elif isinstance(d, str):
        # Escape any existing double quotes in the string
        escaped_str = d.replace('"', '\\"')
        return f'"{escaped_str}"'
    elif isinstance(d, bool):
        return "True" if d else "False"
    elif d is None:
        return "None"
    else:
        return str(d)

# Serialize the dictionary
serialized_geometry_lookup = serialize_dict(geometry_lookup, indent=0)

# Path to the output Python file
output_file_path = r"D:\Documents\E_Plus_2029_py\geomz\geometry_lookup2.py"

# Create the content to write
output_content = f"# geomz/geometry_lookup.py\n\ngeometry_lookup = {serialized_geometry_lookup}\n"

# Write the dictionary to the Python file
try:
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(output_content)
    print(f"geometry_lookup dictionary has been successfully written to {output_file_path}")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")


geometry_lookup dictionary has been successfully written to D:\Documents\E_Plus_2029_py\geomz\geometry_lookup2.py


E envelop Res

In [4]:
import pandas as pd
import os

# Define the path to your Excel file
excel_file_path = r"D:\envelop_res2.xlsx"

# Check if the file exists
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

# List all sheet names in the Excel file
try:
    xls = pd.ExcelFile(excel_file_path)
    sheet_names = xls.sheet_names
    print("Available Sheet Names:")
    for idx, sheet in enumerate(sheet_names):
        print(f"{idx + 1}: {sheet}")
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()


Available Sheet Names:
1: Sheet1


In [9]:
import pandas as pd
import os

# Define the path to your Excel file
#excel_file_path = r"D:\envelop_res2.xlsx"
excel_file_path = r"D:\envelop_nonres3.xlsx"

# Check if the file exists
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Display the original column names
print("Original Column Names:")
print(df.columns.tolist())

# Clean column names: strip spaces and convert to lowercase
df.columns = df.columns.str.strip().str.lower()

# Display the cleaned column names
print("\nCleaned Column Names:")
print(df.columns.tolist())

# Display the first few rows to inspect the data
print("\nFirst 5 Rows of the DataFrame:")
print(df.head())

# Check if all required columns are present
required_columns = [
    'building_function', 'building_type', 'year_range', 'scenario',
    'calibration_stage', 'element', 'area_m2', 'r_value_min',
    'r_value_max', 'u_value_min', 'u_value_max', 'roughness',
    'material_opaque_lookup', 'material_window_lookup',
    'min_wwr', 'max_wwr'
]

missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"\nError: The following required columns are missing from the DataFrame: {missing_columns}")
    exit()
else:
    print("\nAll required columns are present.")

# Replace '-' with NaN
df.replace('-', pd.NA, inplace=True)

# Initialize the dictionary
residential_materials_data = {}

# Group the DataFrame by the desired keys
try:
    grouped = df.groupby(['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage'])
except KeyError as e:
    print(f"\nKeyError: {e}. Please check if all required columns are present and correctly named.")
    exit()

for group_keys, group_df in grouped:
    building_function, building_type, year_range, scenario, calibration_stage = group_keys
    
    # Define the dictionary key tuple
    dict_key = (building_type, year_range, scenario, calibration_stage)
    
    # Initialize the sub-dictionary
    sub_dict = {}
    
    # Extract 'roughness' - assuming it's consistent within the group
    roughness_series = group_df['roughness'].dropna()
    roughness = roughness_series.iloc[0] if not roughness_series.empty else None
    sub_dict['roughness'] = roughness
    
    # Extract 'wwr_range' from 'min_wwr' and 'max_wwr'
    min_wwr = group_df['min_wwr'].dropna().iloc[0] if not group_df['min_wwr'].dropna().empty else None
    max_wwr = group_df['max_wwr'].dropna().iloc[0] if not group_df['max_wwr'].dropna().empty else None
    if min_wwr is not None and max_wwr is not None:
        sub_dict['wwr_range'] = (float(min_wwr), float(max_wwr))
    else:
        sub_dict['wwr_range'] = (None, None)
    
    # Assign top-level material lookups
    # Attempt to get from 'ground_floor', else first available
    ground_floor_row = group_df[group_df['element'] == 'ground_floor']
    if not ground_floor_row.empty:
        material_opaque_lookup = ground_floor_row['material_opaque_lookup'].dropna().iloc[0]
        material_window_lookup = ground_floor_row['material_window_lookup'].dropna().iloc[0] if not ground_floor_row['material_window_lookup'].dropna().empty else None
    else:
        # Take from the first row
        first_row = group_df.iloc[0]
        material_opaque_lookup = first_row['material_opaque_lookup'] if pd.notna(first_row['material_opaque_lookup']) else None
        material_window_lookup = first_row['material_window_lookup'] if pd.notna(first_row['material_window_lookup']) else None
    
    sub_dict['material_opaque_lookup'] = material_opaque_lookup
    sub_dict['material_window_lookup'] = material_window_lookup
    
    # Iterate through each element in the group to add sub-elements
    for _, row in group_df.iterrows():
        element = row['element']
        
        # Initialize the element's sub-dictionary
        element_dict = {}
        
        # Assign area_m2
        area_m2 = row['area_m2']
        if pd.notna(area_m2):
            element_dict['area_m2'] = float(area_m2)
        else:
            element_dict['area_m2'] = None
        
        # Assign R_value_range if available
        R_min = row['r_value_min']
        R_max = row['r_value_max']
        if pd.notna(R_min) and pd.notna(R_max):
            element_dict['R_value_range'] = (float(R_min), float(R_max))
        elif pd.notna(R_min):
            element_dict['R_value_range'] = (float(R_min), float(R_min))
        else:
            element_dict['R_value_range'] = (None, None)
        
        # Assign U_value_range if available
        U_min = row['u_value_min']
        U_max = row['u_value_max']
        if pd.notna(U_min) and pd.notna(U_max):
            element_dict['U_value_range'] = (float(U_min), float(U_max))
        elif pd.notna(U_min):
            element_dict['U_value_range'] = (float(U_min), float(U_min))
        else:
            element_dict['U_value_range'] = (None, None)
        
        # Assign material lookups based on the element
        material_opaque = row['material_opaque_lookup'] if pd.notna(row['material_opaque_lookup']) else None
        material_window = row['material_window_lookup'] if pd.notna(row['material_window_lookup']) else None
        
        if material_opaque:
            element_dict['material_opaque_lookup'] = material_opaque
        if material_window:
            element_dict['material_window_lookup'] = material_window
        
        # Assign the element dictionary to the sub_dict
        sub_dict[element] = element_dict
    
    # Assign the sub_dict to the main dictionary
    residential_materials_data[dict_key] = sub_dict

# Now, residential_materials_data is populated with all the data

# Optionally, you can pretty-print the dictionary to verify
# import pprint
# pprint.pprint(residential_materials_data)

# Define the path to save the Python file
#output_python_file = r"fenez\data_materials_residential2.py"
output_python_file = r"fenez\data_materials_residential2.py"

# Prepare the content to write
with open(output_python_file, 'w') as f:
    f.write("residential_materials_data = {\n")
    for key, value in residential_materials_data.items():
        f.write(f"    {key}: {{\n")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, dict):
                # For nested dictionaries
                f.write(f"        \"{sub_key}\": {{\n")
                for inner_key, inner_val in sub_value.items():
                    if isinstance(inner_val, tuple):
                        f.write(f"            \"{inner_key}\": {inner_val},\n")
                    elif isinstance(inner_val, (float, int)):
                        f.write(f"            \"{inner_key}\": {inner_val},\n")
                    elif inner_val is None:
                        f.write(f"            \"{inner_key}\": None,\n")
                    else:
                        f.write(f"            \"{inner_key}\": \"{inner_val}\",\n")
                f.write("        },\n")
            elif isinstance(sub_value, tuple):
                f.write(f"        \"{sub_key}\": {sub_value},\n")
            elif isinstance(sub_value, (float, int)):
                f.write(f"        \"{sub_key}\": {sub_value},\n")
            elif sub_value is None:
                f.write(f"        \"{sub_key}\": None,\n")
            else:
                f.write(f"        \"{sub_key}\": \"{sub_value}\",\n")
        f.write("    },\n\n")
    f.write("}\n")

print(f"\nresidential_materials_data successfully exported to {output_python_file}")


Original Column Names:
['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage', 'element', 'area_m2', 'R_value_min', 'R_value_max', 'U_value_min', 'U_value_max', 'roughness', 'material_opaque_lookup', 'material_window_lookup', 'min_wwr', 'max_wwr']

Cleaned Column Names:
['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage', 'element', 'area_m2', 'r_value_min', 'r_value_max', 'u_value_min', 'u_value_max', 'roughness', 'material_opaque_lookup', 'material_window_lookup', 'min_wwr', 'max_wwr']

First 5 Rows of the DataFrame:
  building_function     building_type      year_range   scenario  \
0   Non-Residential  Meeting Function  2015 and later  scenario1   
1   Non-Residential  Meeting Function  2015 and later  scenario1   
2   Non-Residential  Meeting Function  2015 and later  scenario1   
3   Non-Residential  Meeting Function  2015 and later  scenario1   
4   Non-Residential  Meeting Function  2015 and later  scenario1   

  c

In [2]:
import pandas as pd
import os

excel_file_path = r"materials.xlsx"
output_python_file = r"fenez\materials_lookup_generated.py"

# 1) Read the Excel
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Clean columns
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# Initialize the final dictionary
material_lookup = {}

def read_range(row, col_min, col_max):
    """
    Returns:
      - (val_min, val_max) if at least one is not null,
      - None if both are blank.
    """
    # Safety check: only attempt if columns exist
    if col_min not in row or col_max not in row:
        return None
    
    val_min = row[col_min]
    val_max = row[col_max]
    
    # If both are NaN -> skip (None return)
    if pd.isna(val_min) and pd.isna(val_max):
        return None
    
    # Convert numeric if not None
    val_min = float(val_min) if pd.notna(val_min) else None
    val_max = float(val_max) if pd.notna(val_max) else None
    
    # If one is None, it means only the other is set
    # You could store (val_min, val_min) if you prefer that behavior
    return (val_min, val_max)

for idx, row in df.iterrows():
    # The material name becomes the key
    mat_name = str(row['name']).strip()
    
    mat_dict = {}

    # -----------------------
    # SINGLE-VALUE PROPERTIES
    # -----------------------
    # For single columns, only add if not blank.
    if 'obj_type' in row and pd.notna(row['obj_type']):
        mat_dict['obj_type'] = str(row['obj_type'])

    if 'roughness' in row and pd.notna(row['roughness']):
        mat_dict['Roughness'] = str(row['roughness'])
    
    # IR_Transmittance (example numeric field):
    if 'ir_transmittance' in row and pd.notna(row['ir_transmittance']):
        # Attempt numeric conversion if appropriate
        try:
            mat_dict['IR_Transmittance'] = float(row['ir_transmittance'])
        except ValueError:
            mat_dict['IR_Transmittance'] = str(row['ir_transmittance'])
    
    # Optical_Data_Type (string, e.g. "SpectralAverage"):
    if 'optical_data_type' in row and pd.notna(row['optical_data_type']):
        mat_dict['Optical_Data_Type'] = str(row['optical_data_type'])

    # Solar_Diffusing (could be "No", "Yes", etc.)
    if 'solar_diffusing' in row and pd.notna(row['solar_diffusing']):
        mat_dict['Solar_Diffusing'] = str(row['solar_diffusing'])

    # -----------------------
    # RANGE-BASED PROPERTIES
    # -----------------------
    # Example: Thickness_range
    thickness_val = read_range(row, 'thickness_min', 'thickness_max')
    if thickness_val is not None:
        mat_dict['Thickness_range'] = thickness_val
    
    # Example: Conductivity_range
    conductivity_val = read_range(row, 'conductivity_min', 'conductivity_max')
    if conductivity_val is not None:
        mat_dict['Conductivity_range'] = conductivity_val
    
    # Repeat for others: Density, Specific_Heat, etc.
    density_val = read_range(row, 'density_min', 'density_max')
    if density_val is not None:
        mat_dict['Density_range'] = density_val

    specific_heat_val = read_range(row, 'specific_heat_min', 'specific_heat_max')
    if specific_heat_val is not None:
        mat_dict['Specific_Heat_range'] = specific_heat_val

    thermal_abs_val = read_range(row, 'thermal_absorptance_min', 'thermal_absorptance_max')
    if thermal_abs_val is not None:
        mat_dict['Thermal_Absorptance_range'] = thermal_abs_val

    solar_abs_val = read_range(row, 'solar_absorptance_min', 'solar_absorptance_max')
    if solar_abs_val is not None:
        mat_dict['Solar_Absorptance_range'] = solar_abs_val

    visible_abs_val = read_range(row, 'visible_absorptance_min', 'visible_absorptance_max')
    if visible_abs_val is not None:
        mat_dict['Visible_Absorptance_range'] = visible_abs_val

    # Example for glazing properties:
    solar_trans_val = read_range(row, 'solar_transmittance_min', 'solar_transmittance_max')
    if solar_trans_val is not None:
        mat_dict['Solar_Transmittance_range'] = solar_trans_val
    
    front_solar_ref_val = read_range(row, 'front_solar_reflectance_min', 'front_solar_reflectance_max')
    if front_solar_ref_val is not None:
        mat_dict['Front_Solar_Reflectance_range'] = front_solar_ref_val
    
    back_solar_ref_val = read_range(row, 'back_solar_reflectance_min', 'back_solar_reflectance_max')
    if back_solar_ref_val is not None:
        mat_dict['Back_Solar_Reflectance_range'] = back_solar_ref_val

    visible_trans_val = read_range(row, 'visible_transmittance_min', 'visible_transmittance_max')
    if visible_trans_val is not None:
        mat_dict['Visible_Transmittance_range'] = visible_trans_val

    front_vis_ref_val = read_range(row, 'front_visible_reflectance_min', 'front_visible_reflectance_max')
    if front_vis_ref_val is not None:
        mat_dict['Front_Visible_Reflectance_range'] = front_vis_ref_val

    back_vis_ref_val = read_range(row, 'back_visible_reflectance_min', 'back_visible_reflectance_max')
    if back_vis_ref_val is not None:
        mat_dict['Back_Visible_Reflectance_range'] = back_vis_ref_val

    front_ir_val = read_range(row, 'front_ir_emissivity_min', 'front_ir_emissivity_max')
    if front_ir_val is not None:
        mat_dict['Front_IR_Emissivity_range'] = front_ir_val

    back_ir_val = read_range(row, 'back_ir_emissivity_min', 'back_ir_emissivity_max')
    if back_ir_val is not None:
        mat_dict['Back_IR_Emissivity_range'] = back_ir_val

    dirt_factor_val = read_range(row, 'dirt_correction_factor_min', 'dirt_correction_factor_max')
    if dirt_factor_val is not None:
        mat_dict['Dirt_Correction_Factor_range'] = dirt_factor_val
    
    # Finally, assign to the main dictionary
    # Skip entirely if mat_name is blank or if there's no data
    if mat_name and mat_dict:
        material_lookup[mat_name] = mat_dict

# Write to Python file
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("# This file is automatically generated. Do not edit manually.\n")
    f.write("material_lookup = {\n")
    for mat_key, mat_val in material_lookup.items():
        f.write(f'    "{mat_key}": {{\n')
        for prop_key, prop_val in mat_val.items():
            if isinstance(prop_val, tuple):
                f.write(f'        "{prop_key}": {prop_val},\n')
            elif prop_val is None:
                f.write(f'        "{prop_key}": None,\n')
            elif isinstance(prop_val, str):
                f.write(f'        "{prop_key}": "{prop_val}",\n')
            else:
                f.write(f'        "{prop_key}": {prop_val},\n')
        f.write("    },\n")
    f.write("}\n")

print(f"material_lookup dictionary successfully exported to {output_python_file}")


material_lookup dictionary successfully exported to fenez\materials_lookup_generated.py


E Elec

In [4]:
import pandas as pd
import os

# ---------------------------------------------------------
# 1) Input Excel and Output Python file
# ---------------------------------------------------------
excel_file_path = r"elec_schedules.xlsx"
output_python_file = r"elec\schedules_lookup2.py"

# ---------------------------------------------------------
# 2) Read the Excel, do some checks
# ---------------------------------------------------------
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Normalize column names
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN if needed
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# ---------------------------------------------------------
# 3) Validate that the essential columns are present
# ---------------------------------------------------------
required_cols = ['building_category', 'building_subtype', 'day_type', 'start_hour', 'end_hour', 'fraction']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    print(f"Error: Missing required columns in the Excel: {missing}")
    exit()

# We can fill NaN or skip rows that are incomplete, depending on your preference:
df.dropna(subset=required_cols, inplace=True)  # remove rows missing any required col

# ---------------------------------------------------------
# 4) Build the SCHEDULE_DEFINITIONS dictionary
# ---------------------------------------------------------
SCHEDULE_DEFINITIONS = {}

# Group by building_category + building_subtype
grouped = df.groupby(['building_category', 'building_subtype'])

for (category, subtype), group_df in grouped:
    # If this category is not yet in the dictionary, initialize it
    if category not in SCHEDULE_DEFINITIONS:
        SCHEDULE_DEFINITIONS[category] = {}
    
    # We'll collect schedules for each day_type in a sub_dict
    sub_dict = {}
    
    # Group again by day_type (weekday, weekend, etc.)
    for day_type, day_df in group_df.groupby('day_type'):
        # We'll build a list of (start_hour, end_hour, fraction)
        schedule_list = []
        
        # Sort day_df by start_hour if you want them in chronological order
        day_df = day_df.sort_values(by='start_hour')
        
        for _, row in day_df.iterrows():
            start_hr = float(row['start_hour'])
            end_hr   = float(row['end_hour'])
            fract    = float(row['fraction'])
            
            # You can skip rows with fraction=0 or negative hours if needed
            # We'll assume all valid if they exist:
            schedule_list.append((start_hr, end_hr, fract))
        
        sub_dict[day_type] = schedule_list
    
    # Assign to SCHEDULE_DEFINITIONS
    SCHEDULE_DEFINITIONS[category][subtype] = sub_dict

# ---------------------------------------------------------
# 5) Write to a Python file
# ---------------------------------------------------------
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("# This file is automatically generated. Do not edit manually.\n")
    f.write("SCHEDULE_DEFINITIONS = {\n")
    for cat_key, cat_val in SCHEDULE_DEFINITIONS.items():
        f.write(f'    "{cat_key}": {{\n')
        
        for subtype_key, subtype_val in cat_val.items():
            f.write(f'        "{subtype_key}": {{\n')
            
            # subtype_val is a dict of day_types -> list of tuples
            for day_key, day_list in subtype_val.items():
                f.write(f'            "{day_key}": [\n')
                
                # Write each tuple, e.g., (0, 6, 0.05)
                for sched_tuple in day_list:
                    f.write(f'                {sched_tuple},\n')
                
                f.write("            ],\n")  # end of day_key list
            f.write("        },\n")  # end of subtype
        f.write("    },\n")  # end of category
    f.write("}\n")

print(f"SCHEDULE_DEFINITIONS dictionary successfully exported to {output_python_file}")


SCHEDULE_DEFINITIONS dictionary successfully exported to elec\schedules_lookup2.py


In [6]:
import pandas as pd
import os

# Path to your Excel file
excel_file_path = r"lighting_lookup.xlsx"

# Path where you want to output the generated .py file
output_python_file = r"Elec\lighting_lookup_generated.py"

def read_range(row, min_col, max_col):
    """
    Returns a tuple (val_min, val_max) if either is not blank;
    returns None if both are blank.
    """
    val_min = row[min_col] if min_col in row and pd.notna(row[min_col]) else None
    val_max = row[max_col] if max_col in row and pd.notna(row[max_col]) else None
    
    if val_min is None and val_max is None:
        return None  # skip if both blank
    
    # Convert to float if not None
    val_min = float(val_min) if val_min is not None else None
    val_max = float(val_max) if val_max is not None else None
    return (val_min, val_max)

# 1) Read the Excel file
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Clean column names
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# 2) Build the lighting_lookup dictionary
lighting_lookup = {}

# We expect each row to have 'scenario', 'building_function', 'building_subtype'
# Then the numeric ranges:
#   lights_wm2_min / lights_wm2_max
#   parasitic_wm2_min / parasitic_wm2_max
#   td_min / td_max
#   tn_min / tn_max

# Iterate over each row in the dataframe
for idx, row in df.iterrows():
    # Scenario
    scenario = str(row['scenario']).strip() if pd.notna(row['scenario']) else None
    if not scenario:
        continue  # skip if no scenario found
    
    # Building Function
    building_func = str(row['building_function']).strip() if pd.notna(row['building_function']) else None
    if not building_func:
        continue  # skip if no building_function found
    
    # Building Subtype
    building_subtype = str(row['building_subtype']).strip() if pd.notna(row['building_subtype']) else None
    if not building_subtype:
        continue  # skip if no building_subtype found
    
    # Initialize the dictionary placeholders if they don't exist
    if scenario not in lighting_lookup:
        lighting_lookup[scenario] = {}
    if building_func not in lighting_lookup[scenario]:
        lighting_lookup[scenario][building_func] = {}
    
    # Create a sub-dict for this building_subtype
    sub_dict = {}
    
    # read_range for the 4 possible ranges
    lw_range = read_range(row, 'lights_wm2_min', 'lights_wm2_max')
    if lw_range is not None:
        sub_dict['LIGHTS_WM2_range'] = lw_range
    
    pwr_range = read_range(row, 'parasitic_wm2_min', 'parasitic_wm2_max')
    if pwr_range is not None:
        sub_dict['PARASITIC_WM2_range'] = pwr_range

    td_range = read_range(row, 'td_min', 'td_max')
    if td_range is not None:
        sub_dict['tD_range'] = td_range

    tn_range = read_range(row, 'tn_min', 'tn_max')
    if tn_range is not None:
        sub_dict['tN_range'] = tn_range
    
    # Assign this sub_dict if it has anything
    if sub_dict:
        lighting_lookup[scenario][building_func][building_subtype] = sub_dict

# 3) Write the dictionary to a Python file
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("\"\"\"Automatically generated lighting_lookup. Do not edit manually.\"\"\"\n\n")
    f.write("lighting_lookup = {\n")
    
    for scenario_key, scenario_val in lighting_lookup.items():
        f.write(f'    "{scenario_key}": {{\n')
        
        for func_key, func_val in scenario_val.items():
            f.write(f'        "{func_key}": {{\n')
            
            for subtype_key, subtype_val in func_val.items():
                f.write(f'            "{subtype_key}": {{\n')
                
                for prop_key, prop_val in subtype_val.items():
                    # If it's a tuple (range)
                    if isinstance(prop_val, tuple):
                        f.write(f'                "{prop_key}": {prop_val},\n')
                    else:
                        f.write(f'                "{prop_key}": {prop_val},\n')
                
                f.write("            },\n")
            
            f.write("        },\n")
        
        f.write("    },\n")
    
    f.write("}\n")

print(f"lighting_lookup dictionary successfully exported to {output_python_file}")


lighting_lookup dictionary successfully exported to Elec\lighting_lookup_generated.py


# EPW

In [7]:
import pandas as pd
import os

# Path to your Excel file
excel_file_path = r"epw_lookup.xlsx"

# Path where you want to output the generated Python file
output_python_file = r"epw\epw_lookup_generated.py"

# 1) Read the Excel
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Clean column names
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# 2) Build the epw_lookup list
epw_lookup = []

for _, row in df.iterrows():
    # Make sure we have a valid file_path
    file_path = row['file_path'] if 'file_path' in row and pd.notna(row['file_path']) else None
    if not file_path:
        # Skip row if no file_path
        continue
    
    # year, lat, lon (optional numeric conversions)
    year_val = None
    if 'year' in row and pd.notna(row['year']):
        # Attempt to cast to int
        try:
            year_val = int(row['year'])
        except ValueError:
            pass  # If invalid, stays None or skip
    
    lat_val = None
    if 'lat' in row and pd.notna(row['lat']):
        try:
            lat_val = float(row['lat'])
        except ValueError:
            pass
    
    lon_val = None
    if 'lon' in row and pd.notna(row['lon']):
        try:
            lon_val = float(row['lon'])
        except ValueError:
            pass

    # Create a dictionary for this row
    epw_entry = {}
    epw_entry["file_path"] = str(file_path)
    
    if year_val is not None:
        epw_entry["year"] = year_val
    if lat_val is not None:
        epw_entry["lat"] = lat_val
    if lon_val is not None:
        epw_entry["lon"] = lon_val
    
    # If you have more columns (e.g., "climate_zone"), handle them similarly:
    # climate_zone = row['climate_zone'] if 'climate_zone' in row and pd.notna(row['climate_zone']) else None
    # if climate_zone:
    #     epw_entry["climate_zone"] = str(climate_zone)
    
    # Append to epw_lookup if something is valid
    if epw_entry:
        epw_lookup.append(epw_entry)

# 3) Write the list to a Python file
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("# Automatically generated epw_lookup. Do not edit manually.\n\n")
    f.write("epw_lookup = [\n")
    
    for entry in epw_lookup:
        f.write("    {\n")
        for k, v in entry.items():
            # If string, wrap in quotes
            if isinstance(v, str):
                f.write(f'        "{k}": "{v}",\n')
            else:
                f.write(f'        "{k}": {v},\n')
        f.write("    },\n")
    
    f.write("]\n")

print(f"epw_lookup list successfully exported to {output_python_file}")


epw_lookup list successfully exported to epw\epw_lookup_generated.py


# Ventilation


In [8]:
import pandas as pd
import os

excel_file_path = r"ventilation_lookup_data.xlsx"
output_python_file = r"ventilation\ventilation_lookup_generated.py"

def read_range(row):
    """
    Reads range_min, range_max from a row.
    Returns:
      - (val_min, val_max) if at least one is not blank,
      - None if both are blank.
    If only one is present, returns (val, val).
    """
    val_min = row.get('range_min', None)
    val_max = row.get('range_max', None)
    
    # If both are completely blank, return None
    if pd.isna(val_min) and pd.isna(val_max):
        return None
    
    # Convert to float if not blank
    min_float = float(val_min) if pd.notna(val_min) else None
    max_float = float(val_max) if pd.notna(val_max) else None
    
    # If only one is provided, use the same value for both
    if min_float is not None and max_float is None:
        max_float = min_float
    elif max_float is not None and min_float is None:
        min_float = max_float
    
    return (min_float, max_float)

# 1) Read the Excel
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Clean columns
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# 2) Build the ventilation_lookup dictionary
ventilation_lookup = {}

for _, row in df.iterrows():
    # scenario
    scenario = row.get('scenario', None)
    if pd.isna(scenario):
        continue
    scenario = str(scenario).strip()
    
    # calibration_stage
    stage = row.get('calibration_stage', None)
    if pd.isna(stage):
        continue
    stage = str(stage).strip()
    
    # param_category (e.g. "residential_infiltration_range", "fan_pressure_range", etc.)
    category = row.get('param_category', None)
    if pd.isna(category):
        continue
    category = str(category).strip()
    
    # read_range
    rng = read_range(row)
    if rng is None:
        # skip if both min & max blank
        continue
    
    # Create the scenario/stage if not exist
    if scenario not in ventilation_lookup:
        ventilation_lookup[scenario] = {}
    if stage not in ventilation_lookup[scenario]:
        ventilation_lookup[scenario][stage] = {}
    
    # param_subkey (e.g. "two_and_a_half_story_house", "A", "1900-2000", or blank)
    subkey = row.get('param_subkey', None)
    subkey = None if pd.isna(subkey) else str(subkey).strip()
    
    # param_nested_key (e.g. "f_ctrl_range"), or blank
    nested_key = row.get('param_nested_key', None)
    nested_key = None if pd.isna(nested_key) else str(nested_key).strip()
    
    # Access the dictionary for this category (create if not exist)
    if category not in ventilation_lookup[scenario][stage]:
        # If we suspect it’s a dictionary of subkeys, we store {} initially
        ventilation_lookup[scenario][stage][category] = {}
    
    # We have 3 possible ways to store the range:
    # 1) If subkey is None -> store the tuple directly: ventilation_lookup[scenario][stage][category] = (..,..)
    # 2) If subkey is present but nested_key is None -> store the tuple at that subkey
    #    ventilation_lookup[scenario][stage][category][subkey] = (.., ..)
    # 3) If subkey AND nested_key are present -> store a dictionary { nested_key: (.., ..) }
    #    ventilation_lookup[scenario][stage][category][subkey][nested_key] = (.., ..)

    if subkey is None:
        # Means we directly attach the tuple to param_category
        ventilation_lookup[scenario][stage][category] = rng
    else:
        # We have a subkey, ensure it’s a dictionary
        if not isinstance(ventilation_lookup[scenario][stage][category], dict):
            # If category was previously set to a tuple, that’s a conflict
            # If needed, you could handle or raise an error. Here we just override.
            ventilation_lookup[scenario][stage][category] = {}
        
        if nested_key is None:
            # Just store the tuple at [subkey]
            ventilation_lookup[scenario][stage][category][subkey] = rng
        else:
            # We store at [subkey][nested_key] => rng
            if subkey not in ventilation_lookup[scenario][stage][category]:
                ventilation_lookup[scenario][stage][category][subkey] = {}
            
            ventilation_lookup[scenario][stage][category][subkey][nested_key] = rng

# 3) Write the dictionary to a Python file
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("\"\"\"Automatically generated ventilation_lookup. Do not edit manually.\"\"\"\n\n")
    f.write("ventilation_lookup = {\n")
    
    # First-level: scenario
    for scenario_key, scenario_val in ventilation_lookup.items():
        f.write(f'    "{scenario_key}": {{\n')
        
        # second-level: calibration_stage
        for stage_key, stage_val in scenario_val.items():
            f.write(f'        "{stage_key}": {{\n')
            
            # third-level: param_category (string -> could be dict or tuple)
            for cat_key, cat_val in stage_val.items():
                f.write(f'            "{cat_key}": ')
                
                # If it’s a tuple, we can write it directly. If it’s a dict, we must recurse or do a manual dict format.
                if isinstance(cat_val, tuple):
                    # e.g. (0.7, 0.8)
                    f.write(f"{cat_val},\n")
                else:
                    # We assume it’s a dict of subkeys => can be nested
                    f.write("{\n")
                    
                    for subkey_key, subkey_val in cat_val.items():
                        # subkey_val could be a tuple or another dict
                        f.write(f'                "{subkey_key}": ')
                        if isinstance(subkey_val, tuple):
                            f.write(f"{subkey_val},\n")
                        elif isinstance(subkey_val, dict):
                            # e.g. "A": {"f_ctrl_range": (0.9, 1.0)}
                            f.write("{\n")
                            for nk, nk_val in subkey_val.items():
                                f.write(f'                    "{nk}": {nk_val},\n')
                            f.write("                },\n")
                        else:
                            f.write(f"{subkey_val},\n")  # fallback
                    f.write("            },\n")
            
            f.write("        },\n")
        
        f.write("    },\n")
    
    f.write("}\n")

print(f"ventilation_lookup dictionary successfully exported to {output_python_file}")


ventilation_lookup dictionary successfully exported to ventilation\ventilation_lookup_generated.py


# groundtemp

In [9]:
import pandas as pd
import os

# Path to your Excel file
excel_file_path = r"groundtemp_lookup.xlsx"

# Path to the Python file you want to generate
output_python_file = r"tempground\groundtemp_lookup_generated.py"

def read_range(row):
    """
    Reads temp_min, temp_max from the row.
    Returns:
      - (val_min, val_max) if at least one is not blank,
      - None if both are blank.
    If only one is provided, uses the same value for both.
    """
    tmin = row.get('temp_min', None)
    tmax = row.get('temp_max', None)
    
    # If both blank => skip
    if pd.isna(tmin) and pd.isna(tmax):
        return None
    
    # Convert to float if not None
    min_val = float(tmin) if pd.notna(tmin) else None
    max_val = float(tmax) if pd.notna(tmax) else None
    
    # If only one is provided, set both to the same
    if min_val is not None and max_val is None:
        max_val = min_val
    elif max_val is not None and min_val is None:
        min_val = max_val
    
    return (min_val, max_val)

if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Clean columns
df.columns = df.columns.str.strip().str.lower()

# Replace '-' or '' with NaN
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# 1) Build the groundtemp_lookup dictionary
groundtemp_lookup = {}

for _, row in df.iterrows():
    scenario = row.get('scenario', None)
    month = row.get('month', None)
    
    if pd.isna(scenario) or pd.isna(month):
        continue  # skip if either is blank
    
    # Convert them to strings
    scenario = str(scenario).strip()
    month = str(month).strip()
    
    # read_range for the temperature
    temp_tuple = read_range(row)
    if temp_tuple is None:
        continue  # skip if both temp_min and temp_max are blank
    
    # Create the scenario dict if not present
    if scenario not in groundtemp_lookup:
        groundtemp_lookup[scenario] = {}
    
    # Assign the temperature tuple to that month
    groundtemp_lookup[scenario][month] = temp_tuple

# 2) Write to a Python file
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("\"\"\"Automatically generated groundtemp_lookup. Do not edit manually.\"\"\"\n\n")
    f.write("groundtemp_lookup = {\n")
    
    for scenario_key, months_dict in groundtemp_lookup.items():
        f.write(f'    "{scenario_key}": {{\n')
        for month_key, temp_val in months_dict.items():
            f.write(f'        "{month_key}": {temp_val},\n')
        f.write("    },\n")
    
    f.write("}\n")

print(f"groundtemp_lookup dictionary successfully exported to {output_python_file}")


groundtemp_lookup dictionary successfully exported to tempground\groundtemp_lookup_generated.py


# dhw

In [11]:
import pandas as pd
import os

excel_file_path = r"dhw_lookup.xlsx"
output_python_file = r"DHW\dhw_lookup_generated.py"

def read_range_or_value(row):
    """
    Checks if there's a single_value present.
    Else attempts to read (range_min, range_max).
    Returns one of:
      - float (if single_value is present),
      - tuple (val_min, val_max) if at least one is present,
      - None if everything is blank.
    """
    single_val = row.get('single_value', None)
    if pd.notna(single_val):
        # convert to float
        try:
            return float(single_val)
        except ValueError:
            return single_val  # fallback if it's string
    # else check range_min / range_max
    min_val = row.get('range_min', None)
    max_val = row.get('range_max', None)
    
    if pd.isna(min_val) and pd.isna(max_val):
        return None
    
    # Convert if not None
    min_float = float(min_val) if pd.notna(min_val) else None
    max_float = float(max_val) if pd.notna(max_val) else None
    
    # If only one is present, replicate
    if min_float is not None and max_float is None:
        max_float = min_float
    elif max_float is not None and min_float is None:
        min_float = max_float
    
    return (min_float, max_float)

# 1) Read Excel
if not os.path.exists(excel_file_path):
    print(f"Error: {excel_file_path} not found.")
    exit()

try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading Excel file: {e}")
    exit()

df.columns = df.columns.str.strip().str.lower()
df.replace('-', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)

# 2) Initialize the final dictionary
dhw_lookup = {
    "TABLE_13_1_KWH_PER_M2": {},
    "pre_calibration": {},
    "post_calibration": {}
}

# 3) Populate from Excel
for _, row in df.iterrows():
    section_type = row.get('section_type', None)
    if pd.isna(section_type):
        continue
    section_type = str(section_type).strip()

    # 'key_name': for TABLE_13_1_KWH_PER_M2 => "Meeting Function"
    # for pre/post_calibration => "Residential_SingleFamily_Small", etc.
    key_name = row.get('key_name', None)
    if pd.isna(key_name):
        continue
    key_name = str(key_name).strip()

    # 'subkey_name': usually the property name if in pre/post_cal, blank if in TABLE_13_1_KWH_PER_M2
    subkey_name = row.get('subkey_name', None)
    subkey_name = None if pd.isna(subkey_name) else str(subkey_name).strip()
    
    val = read_range_or_value(row)
    if val is None:
        # skip blank row
        continue
    
    # Insert into dhw_lookup
    if section_type == "TABLE_13_1_KWH_PER_M2":
        # We expect val to be a single float
        # e.g. dhw_lookup["TABLE_13_1_KWH_PER_M2"]["Meeting Function"] = 2.8
        if not isinstance(val, (float, int)):
            # if it's not numeric, store anyway or skip?
            pass
        dhw_lookup["TABLE_13_1_KWH_PER_M2"][key_name] = val
    
    elif section_type in ("pre_calibration", "post_calibration"):
        # We expect a dictionary of subkeys at dhw_lookup["pre_calibration"][key_name]
        # If that key_name dict doesn't exist, create it
        if key_name not in dhw_lookup[section_type]:
            dhw_lookup[section_type][key_name] = {}
        
        # If subkey_name is None => skip or decide what to do
        # but in your example, we always have a property name
        if subkey_name is None:
            # Means we directly store the val in the dictionary?
            # E.g. dhw_lookup["pre_calibration"]["Residential_SingleFamily_Small"] = (..)
            # But typically you want a name, so let's just skip if no subkey_name
            continue
        
        # Store the value
        dhw_lookup[section_type][key_name][subkey_name] = val

# 4) Write out the dictionary
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("# Automatically generated from dhw_lookup_data.xlsx\n\n")
    f.write("dhw_lookup = {\n")
    
    # A) TABLE_13_1_KWH_PER_M2
    f.write('    "TABLE_13_1_KWH_PER_M2": {\n')
    for func_key, func_val in dhw_lookup["TABLE_13_1_KWH_PER_M2"].items():
        # Numeric single value
        f.write(f'        "{func_key}": {func_val},\n')
    f.write("    },\n\n")
    
    # B) pre_calibration
    f.write('    "pre_calibration": {\n')
    for bld_key, bld_val in dhw_lookup["pre_calibration"].items():
        f.write(f'        "{bld_key}": {{\n')
        for prop_key, prop_val in bld_val.items():
            if isinstance(prop_val, tuple):
                f.write(f'            "{prop_key}": {prop_val},\n')
            elif isinstance(prop_val, (float, int)):
                f.write(f'            "{prop_key}": {prop_val},\n')
            else:
                # fallback if it’s a string or something else
                f.write(f'            "{prop_key}": "{prop_val}",\n')
        f.write("        },\n")
    f.write("    },\n\n")
    
    # C) post_calibration
    f.write('    "post_calibration": {\n')
    for bld_key, bld_val in dhw_lookup["post_calibration"].items():
        f.write(f'        "{bld_key}": {{\n')
        for prop_key, prop_val in bld_val.items():
            if isinstance(prop_val, tuple):
                f.write(f'            "{prop_key}": {prop_val},\n')
            elif isinstance(prop_val, (float, int)):
                f.write(f'            "{prop_key}": {prop_val},\n')
            else:
                f.write(f'            "{prop_key}": "{prop_val}",\n')
        f.write("        },\n")
    f.write("    },\n")
    
    f.write("}\n")

print(f"dhw_lookup dictionary successfully exported to {output_python_file}")


dhw_lookup dictionary successfully exported to DHW\dhw_lookup_generated.py


# Geometry with new intersssssssssssss

In [21]:
import pandas as pd

# =============================================================================
# 1) Read the original Excel file into a DataFrame
# =============================================================================
df = pd.read_excel("D:/envelop_nonres3.xlsx")

# =============================================================================
# 2) Define the columns that define unique groups
#    For each unique combo of these, we'll add new rows
# =============================================================================
group_cols = [
    'building_function',
    'building_type',
    'year_range',
    'scenario',
    'calibration_stage'
]

# =============================================================================
# 3) Identify all unique combinations of those columns
# =============================================================================
unique_groups = df[group_cols].drop_duplicates()

# =============================================================================
# 4) Define the new elements (rows) you want to add, with default values
#    Adjust values as needed for your scenario
# =============================================================================
new_elements = [
    {
        'element': 'interior_wall',
        'area_m2': 50,                 # Example default area
        'R_value_min': 1.5,            # Example default R-value
        'R_value_max': 1.5,
        'U_value_min': 0.67,
        'U_value_max': 0.67,
        'roughness': 'MediumRough',
        'material_opaque_lookup': 'Gypsum_12mm',
        'material_window_lookup': '',  # interior walls usually have no window
        'min_wwr': 0.0,                # interior walls usually no fenestration
        'max_wwr': 0.0
    },
    {
        'element': 'inter_floor',
        'area_m2': 60,                 # Example default area
        'R_value_min': 2.0,            # Example default R-value
        'R_value_max': 2.0,
        'U_value_min': 0.5,
        'U_value_max': 0.5,
        'roughness': 'MediumRough',
        'material_opaque_lookup': 'Concrete_100mm',
        'material_window_lookup': '',  # interior floor usually no window
        'min_wwr': 0.0,
        'max_wwr': 0.0
    }
]

# =============================================================================
# 5) Build a DataFrame (df_new) with the new rows for each unique group
# =============================================================================
new_rows = []
for _, row in unique_groups.iterrows():
    for elem_dict in new_elements:
        new_row = {}
        # Copy over the group columns
        for col in group_cols:
            new_row[col] = row[col]
        # Add the new element defaults
        for key, val in elem_dict.items():
            new_row[key] = val
        new_rows.append(new_row)

df_new = pd.DataFrame(new_rows)

# =============================================================================
# 6) Check for duplicates using only the identifying columns 
#    (group_cols + ['element'])
#    so we only add rows if that group+element combo does not exist in original
# =============================================================================

# (a) Create a "reduced" version of the original DataFrame to detect duplicates
group_cols_with_element = group_cols + ['element']
df_reduced = df[group_cols_with_element].drop_duplicates()

# (b) Merge df_new with df_reduced on these identifying columns
df_merged = pd.merge(
    df_new,
    df_reduced,
    on=group_cols_with_element,
    how='left',
    indicator=True
)

# (c) Keep only the new rows that aren't found in df_reduced
#     (i.e., rows that are 'left_only')
df_new_unique = df_merged[df_merged['_merge'] == 'left_only']

# Now df_new_unique has only the truly new combos
# It retains all the columns from df_new (area_m2, R_value_min, etc.)

# =============================================================================
# 7) Append these unique new rows to the original DataFrame
# =============================================================================
df_combined = pd.concat([df, df_new_unique.drop(columns='_merge')], ignore_index=True)

# =============================================================================
# 8) Write the final DataFrame to a new Excel file
# =============================================================================
df_combined.to_excel("D:/envelop_nonres5.xlsx", index=False)

print("Done! Added new 'interior_wall' and 'inter_floor' rows where they didn't exist.")


Done! Added new 'interior_wall' and 'inter_floor' rows where they didn't exist.


In [26]:
import pandas as pd
import os

# Path to your Excel file
excel_file_path = r"D:\envelop_res5.xlsx"

# Check if the file exists
if not os.path.exists(excel_file_path):
    print(f"Error: The file at {excel_file_path} was not found.")
    exit()

# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error reading the Excel file: {e}")
    exit()

# Display the original column names
print("Original Column Names:")
print(df.columns.tolist())

# Clean column names: strip spaces and convert to lowercase
df.columns = df.columns.str.strip().str.lower()

# Display the cleaned column names
print("\nCleaned Column Names:")
print(df.columns.tolist())

# Display the first few rows to inspect the data
print("\nFirst 5 Rows of the DataFrame:")
print(df.head())

# Check if all required columns are present
required_columns = [
    'building_function', 'building_type', 'year_range', 'scenario',
    'calibration_stage', 'element', 'area_m2', 'r_value_min',
    'r_value_max', 'u_value_min', 'u_value_max', 'roughness',
    'material_opaque_lookup', 'material_window_lookup',
    'min_wwr', 'max_wwr'
]

missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"\nError: The following required columns are missing from the DataFrame: {missing_columns}")
    exit()
else:
    print("\nAll required columns are present.")

# Replace '-' (or any other placeholder) with NaN
df.replace('-', pd.NA, inplace=True)

# Initialize the main dictionary
residential_materials_data = {}

# Group the DataFrame by the primary keys
grouped = df.groupby(['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage'])

for group_keys, group_df in grouped:
    building_function, building_type, year_range, scenario, calibration_stage = group_keys
    
    # Dictionary key: (building_type, year_range, scenario, calibration_stage)
    dict_key = (building_type, year_range, scenario, calibration_stage)
    
    # Initialize the sub-dictionary
    sub_dict = {}
    
    #
    # 1) Get top-level values (roughness, wwr_range, fallback materials)
    #
    # We'll use the first row of this group to grab fallback values
    first_row = group_df.iloc[0]
    
    # a) roughness
    roughness = first_row['roughness'] if pd.notna(first_row['roughness']) else None
    sub_dict['roughness'] = roughness
    
    # b) wwr_range
    min_wwr = first_row['min_wwr'] if pd.notna(first_row['min_wwr']) else None
    max_wwr = first_row['max_wwr'] if pd.notna(first_row['max_wwr']) else None
    sub_dict['wwr_range'] = (
        float(min_wwr) if min_wwr else None,
        float(max_wwr) if max_wwr else None
    )
    
    # c) material_opaque_lookup (fallback)
    fallback_opaque = first_row['material_opaque_lookup'] if pd.notna(first_row['material_opaque_lookup']) else None
    sub_dict['material_opaque_lookup'] = fallback_opaque
    
    # d) material_window_lookup (fallback)
    fallback_window = first_row['material_window_lookup'] if pd.notna(first_row['material_window_lookup']) else None
    sub_dict['material_window_lookup'] = fallback_window
    
    #
    # 2) Create entry for each element row in this group
    #
    for _, row in group_df.iterrows():
        element_name = row['element']  # e.g. "exterior_wall", "ground_floor", etc.
        
        element_dict = {}
        
        # area_m2
        if pd.notna(row['area_m2']):
            element_dict['area_m2'] = float(row['area_m2'])
        else:
            element_dict['area_m2'] = None
        
        # R_value_range
        R_min = row['r_value_min']
        R_max = row['r_value_max']
        if pd.notna(R_min) and pd.notna(R_max):
            element_dict['R_value_range'] = (float(R_min), float(R_max))
        elif pd.notna(R_min):
            element_dict['R_value_range'] = (float(R_min), float(R_min))
        else:
            element_dict['R_value_range'] = (None, None)
        
        # U_value_range
        U_min = row['u_value_min']
        U_max = row['u_value_max']
        if pd.notna(U_min) and pd.notna(U_max):
            element_dict['U_value_range'] = (float(U_min), float(U_max))
        elif pd.notna(U_min):
            element_dict['U_value_range'] = (float(U_min), float(U_min))
        else:
            element_dict['U_value_range'] = (None, None)
        
        # material_opaque_lookup: use row’s if present, else fallback
        if pd.notna(row['material_opaque_lookup']):
            element_dict['material_opaque_lookup'] = row['material_opaque_lookup']
        elif fallback_opaque:
            element_dict['material_opaque_lookup'] = fallback_opaque
        
        # material_window_lookup: use row’s if present, else fallback
        if pd.notna(row['material_window_lookup']):
            element_dict['material_window_lookup'] = row['material_window_lookup']
        elif fallback_window:
            element_dict['material_window_lookup'] = fallback_window
        
        # Add this element’s dict into the sub_dict
        sub_dict[element_name] = element_dict
    
    # Assign the sub_dict to the main dictionary
    residential_materials_data[dict_key] = sub_dict

#
# 3) (Optional) Write out to a .py file
#
output_python_file = r"fenez\data_materials_residential3.py"
with open(output_python_file, 'w', encoding='utf-8') as f:
    f.write("residential_materials_data = {\n")
    for key, value in residential_materials_data.items():
        f.write(f"    {key}: {{\n")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, dict):
                # Nested dictionary
                f.write(f"        \"{sub_key}\": {{\n")
                for inner_key, inner_val in sub_value.items():
                    if isinstance(inner_val, tuple):
                        f.write(f"            \"{inner_key}\": {inner_val},\n")
                    elif isinstance(inner_val, (float, int)):
                        f.write(f"            \"{inner_key}\": {inner_val},\n")
                    elif inner_val is None:
                        f.write(f"            \"{inner_key}\": None,\n")
                    else:
                        f.write(f"            \"{inner_key}\": \"{inner_val}\",\n")
                f.write("        },\n")
            elif isinstance(sub_value, tuple):
                f.write(f"        \"{sub_key}\": {sub_value},\n")
            elif isinstance(sub_value, (float, int)):
                f.write(f"        \"{sub_key}\": {sub_value},\n")
            elif sub_value is None:
                f.write(f"        \"{sub_key}\": None,\n")
            else:
                f.write(f"        \"{sub_key}\": \"{sub_value}\",\n")
        f.write("    },\n\n")
    f.write("}\n")

print(f"\nDictionary successfully created and exported to {output_python_file}")


Original Column Names:
['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage', 'element', 'area_m2', 'R_value_min', 'R_value_max', 'U_value_min', 'U_value_max', 'roughness', 'material_opaque_lookup', 'material_window_lookup', 'min_wwr', 'max_wwr']

Cleaned Column Names:
['building_function', 'building_type', 'year_range', 'scenario', 'calibration_stage', 'element', 'area_m2', 'r_value_min', 'r_value_max', 'u_value_min', 'u_value_max', 'roughness', 'material_opaque_lookup', 'material_window_lookup', 'min_wwr', 'max_wwr']

First 5 Rows of the DataFrame:
  building_function building_type year_range   scenario calibration_stage  \
0       residential     Apartment      <1946  scenario1   pre_calibration   
1       residential     Apartment      <1946  scenario1  post_calibration   
2       residential     Apartment      <1946  scenario1   pre_calibration   
3       residential     Apartment      <1946  scenario1  post_calibration   
4       residential     Apa